In [293]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [242]:
cursor=db.companies.aggregate([
    {'$match': 
     {'$and': [ 
        {'category_code': {'$in': ["web","software","games_video"]}},
        {'founded_year':{'$gte':2009}},
        {'deadpooled_year': None },
        {"offices.latitude":{'$ne': None}},
        {"offices.longitude":{'$ne': None}},
        {"number_of_employees":{'$ne': None}},
         
    ]}},
    { '$unwind': "$offices" },
    { '$project' : {
        "name" : 1,
        "category_code": 1,
        "founded_year": 1,
        "description":1,
        "money_raised":1,
        "number_of_employees":1,
        "offices.latitude": 1,
        "offices.longitude":1,
        "offices.country_code":1,
        "_id":0
    }}])

Comp=list(cursor)

In [243]:
import pandas as pd

df = pd.DataFrame(Comp)
df

,category_code,description,founded_year,name,number_of_employees,offices
0,web,Tech Support Community,2013,Fixya,30,"{'country_code': 'USA', 'latitude': 37.566879,..."
1,games_video,,2009,alluc,7,"{'country_code': 'DEU', 'latitude': 53.707739,..."
2,games_video,social gaming platform,2011,Social Gaming Network,100,"{'country_code': 'USA', 'latitude': 37.446823,..."
3,games_video,social gaming platform,2011,Social Gaming Network,100,"{'country_code': 'USA', 'latitude': 34.0698494..."
4,games_video,Educational Games,2009,Crootpad,2,"{'country_code': 'USA', 'latitude': 42.091078,..."
5,software,Financial Engineering Platform,2009,Macroaxis,8,"{'country_code': 'USA', 'latitude': 37.7896292..."
6,games_video,People Powered Radio,2011,Fuzz,6,"{'country_code': 'USA', 'latitude': 37.760524,..."
7,web,Take Control of Your Own Search Results,2009,BrandYourself,22,"{'country_code': 'USA', 'latitude': 42.275263,..."
8,web,College Life Online,2010,CollegeConvo,2,"{'country_code': 'USA', 'latitude': 40.650291,..."
9,software,Big data automotive marketing platform,2012,Carfeine,5,"{'country_code': 'USA', 'latitude': 38.989124,..."


In [244]:
df['lat'] = df.apply(lambda row: row['offices']['latitude'], axis=1)
df['long'] = df.apply(lambda row: row['offices']['longitude'], axis=1)
df['country'] = df.apply(lambda row: row['offices']['country_code'], axis=1)
df['coord'] = df[['lat', 'long']].apply(list, axis=1)
df=df.drop(columns=["offices"])
#df['coordinates']={'type':'Point', "coordinates":['coord']}
df.head()


,category_code,description,founded_year,name,number_of_employees,lat,long,country,coord
0,web,Tech Support Community,2013,Fixya,30,37.566879,-122.323895,USA,"[37.566879, -122.323895]"
1,games_video,,2009,alluc,7,53.707739,10.023246,DEU,"[53.707739, 10.023246]"
2,games_video,social gaming platform,2011,Social Gaming Network,100,37.446823,-122.161523,USA,"[37.446823, -122.161523]"
3,games_video,social gaming platform,2011,Social Gaming Network,100,34.069849,-118.405418,USA,"[34.0698494, -118.4054182]"
4,games_video,Educational Games,2009,Crootpad,2,42.091078,-88.288749,USA,"[42.091078, -88.288749]"


In [245]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[1],array[0]]}
df['position'] = df.apply(lambda r:toGeoJSON(r['coord']),axis=1)
df=df.drop(columns=["coord"])

In [246]:
df.head()

,category_code,description,founded_year,name,number_of_employees,lat,long,country,position
0,web,Tech Support Community,2013,Fixya,30,37.566879,-122.323895,USA,"{'type': 'Point', 'coordinates': [-122.323895,..."
1,games_video,,2009,alluc,7,53.707739,10.023246,DEU,"{'type': 'Point', 'coordinates': [10.023246, 5..."
2,games_video,social gaming platform,2011,Social Gaming Network,100,37.446823,-122.161523,USA,"{'type': 'Point', 'coordinates': [-122.161523,..."
3,games_video,social gaming platform,2011,Social Gaming Network,100,34.069849,-118.405418,USA,"{'type': 'Point', 'coordinates': [-118.4054182..."
4,games_video,Educational Games,2009,Crootpad,2,42.091078,-88.288749,USA,"{'type': 'Point', 'coordinates': [-88.288749, ..."


In [247]:
df.to_json('companies_filtered.json', orient="records", lines=True)

In [282]:
client = MongoClient()
db = client.comp


In [298]:
cursor=db.comp.find()
distance=[]
for i in cursor:
    cursor2 = db.comp.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [i['position']['coordinates'][0],i['position']['coordinates'][1]]},
      "$minDistance": 0,
      "$maxDistance":10000
    }}})
    distance.append(cursor2.count())
print(distance)
    



/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]


[1, 1, 5, 3, 1, 4, 4, 1, 2, 3, 5, 1, 3, 3, 1, 3, 1, 1, 8, 1, 3, 8, 1, 2, 4, 1, 1, 1, 2, 2, 1, 2, 3, 1, 1, 1, 8, 1, 1, 1, 2, 4, 1, 2, 1, 1, 1, 5, 2, 1, 2, 1, 1, 3, 8, 2, 3, 2, 4, 1, 1, 1, 5, 5, 3, 5, 1, 1, 1, 2, 2, 2, 1, 1, 1, 5, 1, 1, 3, 1, 1, 1, 2, 1, 1, 3, 2, 1, 1, 1, 1, 2, 8, 1, 1, 3, 1, 1, 8, 1, 5, 5, 5, 2, 4, 1, 1, 3, 8, 5, 1, 2, 5, 1, 1, 1, 3, 1, 1, 8, 1, 2, 3, 1]
